## 2.2 thesaurus
뜻 : 유의어 사전(동의어, 유의어의 집합)
![word_tree](../deep_learning_2_images/fig%202-1.png)
![word_tree2](../deep_learning_2_images/fig%202-2.png)
단어에 대한 유의어 집합을 형성하고 그에 대한 네트워크까지 짜주면 컴퓨터가 해당 단어를 이해했다고 생각할 수 있다.

### 2.2.1 WordNet
1985년부터 시작한 유의어 획득 단어 네트워크

### 2.2.2 thesaurus의 문제점
수많은 대한 동의어나 유의어의 계층을 형성해도, 시대적 맥락을 따라오지 못할 가능성이 크다(새로 추가된 맥락을 넣어주어야 함). 또한 사람들을 직접 동원하여 수작업으로 thesaurus를 만들어야한다. 마지막으로 단어의 미묘한 차이를 표현할 수 없다.

## 2.3 통계 기반 기법
말뭉치(corpus) : 자연어 처리 연구나 애플리케이션을 염두에 두고 수집된 텍스트 데이터
### 2.3.1 말뭉치 전처리하기

In [2]:
# 텍스트 설정
text = 'You say goodbye and I say hello.'
# text 변형
text = text.lower()
text = text.replace('.', ' .')
print(text)

you say goodbye and i say hello .


In [3]:
# 단어 쪼개기
words = text.split(' ')
print(words)

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']


In [4]:
word_to_id = {}
id_to_word = {}
# 단어 쪼개서 딕션어리 입력
for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

print(id_to_word)
print(word_to_id)

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}
{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}


In [5]:
print(id_to_word[1])

say


In [6]:
print(word_to_id['hello'])

5


In [8]:
import numpy as np
corpus = [word_to_id[w] for w in words] # 반복문을 간단하게 처리하는 기법
corpus = np.array(corpus)
print(corpus)

[0 1 2 3 4 1 5 6]


In [11]:
# 전처리 함수
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')
    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word
# 텍스트 전처리 과정 축약
text = 'You Say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

### 2.3.2 단어의 분산 표현

단어의 표현 즉 단어의 의미를 정확하게 파악할 수 있는 표현 -> 분산 표현(distributional representation)
각 단어의 분산 표현인 벡터의 유사성을 바탕으로 단어 간의 관계를 설정하여 나가면 된다!

### 2.3.3 분포 가설

단어의 의미는 주변 단어에 의해 형성된다. -> 이를 기반으로 분포 가설(distributional hypothesis)
=> 해당 단어에는 의미가 없고, 그 단어가 사용된 맥락이 의미를 형성한다.

ex.) I can drink beer, We drink wine ... 에서는 drink 주위에 음료가 들어갈 가능성이 크다는 것을 알 수가 있다., drink 대신에 guzzle이란 단어가 들어간다면 drink랑 같은 맥락으로 해석도 가능할 것이다.

여기서 맥락은 주변에 놓인 단어를 뜻하며, 좌우 각 몇단어 씩 들어가냐에 따라서 윈도우 크기(window size)를 정한다.
![window_size](../deep_learning_2_images/fig%202-3.png)

### 2.3.4 동시발행 행렬

어떤 단어에 주목했을 때 그 주변에 어떤 단어가 몇 번이나 등장하는 지 세어보는 방법!(= 통계 기반(statistical) 기법)

In [ ]:
# 단어 아이디 list, 어휘 수, 윈도우 크기 넣어서 동시발행 행렬 출력하는 함수

def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size+1):
            left_idx = idx - 1
            right_idx = idx + 1

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id][left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id][right_word_id] += 1

    return co_matrix